## Prepare the tables necessary to calculate motif disruption
1. Get the SNPs to test: in this case I am testing all heterozygous SNPs that were tested for ASE in ChIP-Seq
2. Get the fasta for a 41 bp window (20 bases upstream and downstream), for both alleles
3. Get the results of the 20 most enrichemt motif e from Homer
4. Run the analysis using the script scoreMotifs_m1.R (computationally intense and long)

In [1]:
library (stringr)
home="/home/paola/Family1070/private_output/"
dir="ASE_chip/correlationWithBeta/"
names=c("NKX25", "H3K27AC_IPSC",  "H3K27AC_CM","ATAC_IPSC", "ATAC_CM")  #also names of directories
files= paste('ASE.tested', names, 'txt', sep=".")
setwd(home)
setwd("Motif_analysis_R")

In [2]:
FILES = files[1]
name  = names[1]
#dir.create(name)

In [7]:
ase=read.table(paste(home,dir,FILES,sep=""), sep="\t", header=T)
test=str_split_fixed(ase$variantID, ":", 2)
ase$chr=test[,1]
ase$POS=as.numeric(test[,2])
ase$bed.start=ase$POS-20
ase$bed.end=ase$POS+20

write.table(ase[,c("chr", "bed.start", "bed.end","variantID")],
            paste(name, "all_snps.bed", sep="/"), row.names=F, col.names=F, sep="\t", quote=FALSE)



command= paste("bedtools getfasta -fi /publicdata/hg19_20151104/hg19_sorted.fa -bed  ", 
               name, "/all_snps.bed -fo " , name,  "/all_fasta.tab -name -tab", sep="")

system("module load cardips")
system(command)


  file=paste(name, "all_fasta.tab", sep="/")
  fas=read.table(file)
  
  fas=cbind(fas,ase[,c("refAllele", "altAllele", 'ref_freq', 'combined_fdr')])
  # fas=cbind(fas,ase[,c("refAllele", "altAllele", 'new_maf', 'combined_fdr')]) # colum name of previous file
  colnames(fas)=c("pos","seq","ref","alt",'ref_freq', 'combined_fdr')
  fas[,1:4]=sapply(fas[,1:4], as.character)
  fas1=fas
  
  
  # Commands for writing fasta format
  Xfasta <- character(nrow(fas) * 2)
  Xfasta[c(TRUE, FALSE)] <- paste0(">", fas$pos)
  Xfasta[c(FALSE, TRUE)] <- fas$seq
  writeLines(Xfasta, paste(name,"ref.fasta", sep="/"))

  ###########
  for (i in 1:nrow(fas1)){
   
      substr(fas1[i,"seq"], 20, 20) <- fas1[i,"alt"]
    
  }
  
  # Commands for writing fasta format
  Xfasta <- character(nrow(fas1) * 2)
  Xfasta[c(TRUE, FALSE)] <- paste0(">", fas1$pos)
  Xfasta[c(FALSE, TRUE)] <- fas1$seq
  writeLines(Xfasta, paste(name,"alt.fasta", sep="/"))
  


##### Make table to use in Erin's program
matr=cbind(fas, fas1[,c("seq")])
matr=matr[c(1,3,4,5,6,2,7)]
#adapt column names to erin's script 
colnames(matr)[c(1,6,7)]=c("Variant", "REfSeq", "ALtSeq")
matr$REfSeq<-toupper(matr$REfSeq)
matr$ALtSeq<-toupper(matr$ALtSeq)

write.table(matr, paste(name, "TestMAtr.txt", sep="/"), row.names=F, col.names=T, sep="\t", quote=F)



In [8]:
head(matr)

Variant,ref,alt,ref_freq,combined_fdr,REfSeq,ALtSeq
chr10:100104314,T,C,0.5174881,0.9975985,CTGCCTCCTCCTCCTCCTCTGTGGTTGGGAGATTTCACAT,CTGCCTCCTCCTCCTCCTCCGTGGTTGGGAGATTTCACAT
chr10:100220928,A,T,0.4915662,0.8244251,GAGAGAGATAATCAGACCCATGCCCAATAGAGCAGTGAAT,GAGAGAGATAATCAGACCCTTGCCCAATAGAGCAGTGAAT
chr10:100221081,T,C,0.5220204,1.0000000,AGGGAGCAGGCCCCCTCCCTTTGCTAGAATGACTGAGTGA,AGGGAGCAGGCCCCCTCCCCTTGCTAGAATGACTGAGTGA
chr10:10034807,G,A,0.4623286,0.9635344,TCGCCAAGCAGACAATCACGGAGGCTTAAGGCCGTGCATT,TCGCCAAGCAGACAATCACAGAGGCTTAAGGCCGTGCATT
chr10:100361679,G,T,0.2781205,0.4605427,AGCAAAGGCAAACTTGTTAGCTTGCAAATAGGGTAACTCT,AGCAAAGGCAAACTTGTTATCTTGCAAATAGGGTAACTCT
chr10:10043586,A,T,0.5519033,0.8423266,GTACAAGGTCATCCCCCACAAAGAGTGTGAGCAACCTTAG,GTACAAGGTCATCCCCCACTAAGAGTGTGAGCAACCTTAG


In [46]:
motif_dir_new = paste(name, "analysis","PWMs", sep="/")
dir.create(motif_dir_new)

Warning message in dir.create(motif_dir_new):
"'NKX25/analysis/PWMs' already exists"

In [47]:
motif_dir = paste(home, "Homer_motif_enrichment/", name, "/knownResults/", sep="")

In [52]:
homer_scores = data.frame()
for( i in 1:20){
   
   pwm = read.table(paste(motif_dir, "known",i, ".motif", sep=""), sep="\t", skip=1)
   colnames(pwm)=c('A','C','G','T')
   
    head = read.table(paste(motif_dir, "known",i, ".motif", sep=""), sep="\t", nrows=1, stringsAsFactor=F) 
    homer_scores[i,1]= str_split_fixed(head$V2, "\\(", 2 )[,1]
    homer_scores[i,2]= head$V3
    
    write.table(pwm, paste(motif_dir_new, homer_scores[i,1], sep="/" ),  sep="\t",  row.names=F, quote=F) 
}


write.table(homer_scores, paste(name, "analysis", "HomerScores.txt" , sep="/"), quote= F, col.names=F, row.names=F) 

To run the analysis, qsub the code 2_scoreMotifs_m1.R  --- it takes about an hour per motif.